In [1]:
import numpy as np
import cv2
import tensorflow as tf
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from numpy.random import randn
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import RepeatVector
from matplotlib import pyplot
from tensorflow.keras.utils import plot_model

import time as t
from statistics import mean

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 4GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10240)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
# define the discriminator model
def define_discriminator(image_shape, vector_shape):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # source image input
    in_src_image = Input(shape=image_shape)
    # target image input
    in_target = Input(shape=(vector_shape[0], vector_shape[1]))
    # Vector layer
    n_nodes = (image_shape[0] * image_shape[1] * image_shape[2])
    # LSTM Model
    p = LSTM(200, activation='relu')(in_target)
    #p = LSTM(200, activation=None)(in_target)
    p = Dense(n_nodes)(p)
    # Reshape and mergo to image dimension
    p = Reshape((image_shape[0], image_shape[1], image_shape[2]))(p)
    merged = Concatenate()([p, in_src_image])
    # concatenate images channel-wise
    # C64
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    # C128
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C256
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C512
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # second last output layer
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # patch output
    d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Flatten()(d)
    d = Dropout(0.4)(d)
    out_layer = Dense(1, activation='sigmoid')(d)
    # define model
    model = Model([in_src_image, in_target], out_layer)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model


In [4]:
# define an encoder block
def define_encoder_block(layer_in, n_filters, batchnorm=True):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # add downsampling layer
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same',
    kernel_initializer=init)(layer_in)
    # conditionally add batch normalization
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    # leaky relu activation
    g = LeakyReLU(alpha=0.2)(g)
    return g

In [5]:
# define the standalone generator model
def define_generator(in_shape, vector_shape, latent_dim):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # image input
    in_image = Input(shape=in_shape)
    in_lat = Input(shape=latent_dim)
    gen = LeakyReLU(alpha=0.2)(in_lat)
    # merge image gen and label input
    # encoder model
    e1 = define_encoder_block(in_image, 64, batchnorm=False)
    e2 = define_encoder_block(e1, 128)
    e3 = define_encoder_block(e2, 256)
    e4 = define_encoder_block(e3, 512)
    e5 = define_encoder_block(e4, 512)
    e6 = define_encoder_block(e5, 512)
    e7 = define_encoder_block(e6, 512)
    # bottleneck, no batch norm and relu
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
    b = Activation('relu')(b)
    b = Flatten()(b)
    # Merge latent space variable input with network
    b = Concatenate()([b, gen])
    # Reshape to vector size
    n_nodes = (vector_shape[0] * vector_shape[1])
    b = Dense(n_nodes)(b)
    b = Reshape([vector_shape[0], vector_shape[1]])(b)
    # encoder-decoder LSTM model
    d = LSTM(200, activation='relu')(b)
    d = RepeatVector(vector_shape[0])(d)
    d = LSTM(200, activation='relu', return_sequences=True)(d)
    # output
    out_layer = Dense(3)(d)

    # define model input & output
    model = Model([in_image, in_lat], out_layer)
    return model

In [6]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model, image_shape, vector_shape, latent_dim):
    # make weights in the discriminator not trainable
    for layer in d_model.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False
    # define the source image
    in_src = Input(shape=image_shape)
    in_lat = Input(shape=latent_dim)
    # connect the source image to the generator input
    gen_out = g_model([in_src, in_lat])
    # connect the source input and generator output to the discriminator input
    dis_out = d_model([in_src, gen_out])
    # src image as input, generated image and classification output
    model = Model([in_src, in_lat], [dis_out, gen_out])
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    return model

In [7]:
# load and prepare training images
def load_real_samples(filename):
	# load compressed arrays
	data = load(filename)
	# unpack arrays
	X1, X2 = data['arr_0'], data['arr_1']
	return [X1, X2]

In [8]:
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples):
	# unpack dataset
	trainA, trainB = dataset
	# choose random instances
	ix = randint(0, trainA.shape[0], n_samples)
	# retrieve selected images
	X1, X2 = trainA[ix], trainB[ix]
	# generate 'real' class labels 
	y = ones((n_samples, 1))
	return [X1, X2], y

In [9]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [10]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, n_samples):
    # Generate latent space points
    z_input = generate_latent_points(latent_dim, n_samples)
    # generate fake instance
    X = g_model.predict([samples, z_input])
    # create 'fake' class labels 
    y = zeros((n_samples, 1))
    return X, y

In [11]:
# Overlay trajectories (data_y) to the image (data_x)
def create_trajectory(data_x, data_y, obs_len=10):
    # Calibration parameter to overlay for a 1280x360 resolution image
    K = np.array([[537.023764, 0, 640 , 0], 
                    [0 , 537.023764, 180, 0], 
                    [0, 0, 1, 0]])
    # Rotation matrix to obtain egocentric trajectory
    Rt = np.array([[0.028841, 0.007189, 0.999558, 1.481009],
                    [-0.999575,  0.004514,  0.028809,  0.296583],
                    [ 0.004305,  0.999964, -0.007316, -1.544537],
                    [ 0.      ,  0.      ,  0.      ,  1.      ]])

    # Resize data back to 1280x360
    data_x = cv2.resize(data_x, (1280,360))
    # Add column of ones for rotation matrix multiplication
    data_y = np.hstack((data_y, np.ones((len(data_y),1))))
    # Draw points
    for m in range(obs_len, data_y.shape[0]):
        # Rotation matrix multiplication of trajectory 
        A = np.matmul(np.linalg.inv(Rt), data_y[m, :].reshape(4, 1))
        # Egocentric view of trajectory
        B = np.matmul(K, A)
        # Circle location of trajectories 
        x = int(B[0, 0] * 1.0 / B[2, 0])
        y = int(B[1, 0] * 1.0 / B[2, 0])
        if (x < 0 or x > 1280 - 1 or y > 360 - 1):
            continue
        # Use opencv to overlay trajectories
        data_x = cv2.circle(data_x, (x, y), 3, (0, 0, 255), -1)
    return data_x

In [12]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, dataset, n_samples=1):
    # select a sample of input images
    [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples)
    # generate a batch of fake samples
    X_fakeB, _ = generate_fake_samples(g_model, X_realA, n_samples)
    # scale all pixels from [-1,1] to [0,1]
    X_realA = (X_realA + 1) / 2.0
    # plot real source images
    for i in range(n_samples):
        orig_image = (X_realA[i]* 255).astype(np.uint8)
        orig_image = cv2.resize(orig_image, (1280,360))
        pyplot.subplot(3, n_samples, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(orig_image)
    # plot generated target image
    for i in range(n_samples):
        fake_sample = create_trajectory((X_realA[i]* 255).astype(np.uint8), X_fakeB[i])
        pyplot.subplot(3, n_samples, 1 + n_samples + i)
        pyplot.axis('off')
        pyplot.imshow(fake_sample)
    # plot real target image
    for i in range(n_samples):
        true_sample = create_trajectory((X_realA[i]* 255).astype(np.uint8), X_realB[i])
        pyplot.subplot(3, n_samples, 1 + n_samples*2 + i)
        pyplot.axis('off')
        pyplot.imshow(true_sample)
    # save plot to file
    filename1 = 'plot_%06d.png' % (step+1)
    pyplot.savefig(filename1)
    pyplot.close()
    # save the generator model
    filename2 = 'model_%06d.h5' % (step+1)
    g_model.save(filename2)
    print('>Saved: %s and %s' % (filename1, filename2))

In [13]:
# train models
def train(d_model, g_model, gan_model, dataset, latent_dim, n_epochs=50, n_batch=1):
    # calculate the number of batches per training epoch
    bat_per_epo = int(len(dataset[0]) / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    #for timining
    #n_steps = 25
    times = []
    # manually enumerate epochs
    for i in range(n_steps):
        #start timer
        start = t.time()
        # select a batch of real samples
        [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch)
        # generate a batch of fake samples
        X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_batch)
        # update discriminator for real samples
        d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
        # update discriminator for generated samples
        d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
        X_lat = generate_latent_points(latent_dim, n_batch)
        # update the generator
        g_loss, _, _ = gan_model.train_on_batch([X_realA, X_lat], [y_real, X_realB])
        #end timer
        end = t.time()
        #print(f'time for itr: {end-start} sec')
        times.append(end-start)
        # summarize performance
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
        # summarize model performance
        if (i+1) % int(bat_per_epo) == 0:
            summarize_performance(i, g_model, dataset)
            print('saved')
    print(f'average time per itr: {mean(times)}')

In [14]:
%%time
# load image data
#dataset = load_real_samples('0000.npz')
dataset = load_real_samples('dataset.npz')
print('Loaded', dataset[0].shape, dataset[1].shape)

Loaded (19031, 256, 256, 3) (19031, 40, 3)
CPU times: user 24.9 s, sys: 822 ms, total: 25.7 s
Wall time: 25.7 s


In [15]:
# define input shape based on the loaded dataset
image_shape = dataset[0].shape[1:]
vector_shape = dataset[1].shape[1:]
latent_dim = 512
print(image_shape)
print(vector_shape)

(256, 256, 3)
(40, 3)


In [16]:
# define the models
d_model = define_discriminator(image_shape, vector_shape)
g_model = define_generator(image_shape, vector_shape, latent_dim)

In [17]:
# define the composite model
gan_model = define_gan(g_model, d_model, image_shape, vector_shape, latent_dim)

In [ ]:
train(d_model, g_model, gan_model, dataset, latent_dim)

>1, d1[0.563] d2[4.454] g[233.807]
>2, d1[0.000] d2[1.180] g[415.675]
>3, d1[0.004] d2[0.274] g[696.758]
>4, d1[0.000] d2[0.322] g[393.832]
>5, d1[0.008] d2[0.530] g[543.371]
>6, d1[0.002] d2[0.188] g[481.142]
>7, d1[0.000] d2[0.452] g[647.202]
>8, d1[0.014] d2[0.093] g[372.992]
>9, d1[0.190] d2[1.237] g[242.089]
>10, d1[0.000] d2[1.225] g[853.822]
>11, d1[0.000] d2[1.603] g[676.208]
>12, d1[0.000] d2[0.457] g[821.611]
>13, d1[0.000] d2[0.144] g[860.251]
>14, d1[0.163] d2[0.861] g[210.544]
>15, d1[0.000] d2[0.605] g[336.150]
>16, d1[0.088] d2[1.031] g[144.632]
>17, d1[0.174] d2[0.713] g[93.342]
>18, d1[0.000] d2[0.020] g[665.718]
>19, d1[0.000] d2[0.048] g[363.790]
>20, d1[0.646] d2[0.531] g[257.748]
>21, d1[0.035] d2[5.125] g[2142.792]
>22, d1[0.000] d2[2.120] g[362.201]
>23, d1[0.288] d2[0.044] g[343.972]
>24, d1[0.012] d2[0.006] g[444.213]
>25, d1[0.039] d2[0.485] g[286.982]
>26, d1[0.384] d2[0.236] g[96.363]
>27, d1[0.273] d2[0.476] g[77.382]
>28, d1[0.009] d2[0.066] g[442.452]
>29

>227, d1[0.068] d2[0.161] g[83.739]
>228, d1[0.187] d2[0.006] g[391.158]
>229, d1[0.034] d2[0.002] g[142.766]
>230, d1[0.177] d2[0.001] g[392.801]
>231, d1[0.096] d2[0.005] g[229.812]
>232, d1[0.469] d2[0.001] g[252.225]
>233, d1[0.031] d2[0.008] g[193.721]
>234, d1[0.416] d2[1.455] g[192.975]
>235, d1[0.147] d2[0.006] g[220.223]
>236, d1[0.183] d2[0.699] g[106.209]
>237, d1[0.293] d2[0.343] g[172.529]
>238, d1[0.214] d2[0.263] g[139.686]
>239, d1[0.164] d2[0.117] g[325.507]
>240, d1[0.025] d2[0.081] g[277.661]
>241, d1[0.046] d2[0.719] g[115.123]
>242, d1[0.101] d2[0.663] g[140.420]
>243, d1[0.090] d2[0.607] g[133.821]
>244, d1[0.308] d2[0.255] g[240.638]
>245, d1[0.437] d2[0.006] g[114.152]
>246, d1[0.837] d2[0.409] g[246.431]
>247, d1[0.164] d2[0.919] g[174.731]
>248, d1[0.484] d2[0.003] g[148.264]
>249, d1[0.420] d2[0.002] g[449.662]
>250, d1[0.353] d2[0.003] g[334.704]
>251, d1[0.230] d2[0.001] g[209.975]
>252, d1[0.285] d2[0.011] g[152.959]
>253, d1[0.531] d2[0.305] g[168.461]
>2

>450, d1[0.353] d2[0.225] g[143.404]
>451, d1[0.388] d2[0.617] g[221.595]
>452, d1[0.540] d2[0.238] g[195.509]
>453, d1[0.179] d2[0.151] g[206.263]
>454, d1[0.254] d2[0.446] g[96.765]
>455, d1[0.147] d2[0.387] g[193.710]
>456, d1[0.613] d2[0.209] g[242.826]
>457, d1[0.384] d2[0.332] g[105.914]
>458, d1[0.592] d2[0.347] g[290.421]
>459, d1[0.332] d2[0.327] g[31.154]
>460, d1[0.551] d2[0.127] g[180.958]
>461, d1[0.381] d2[0.154] g[268.377]
>462, d1[0.401] d2[0.226] g[56.071]
>463, d1[0.395] d2[0.121] g[136.107]
>464, d1[0.653] d2[0.462] g[474.873]
>465, d1[0.543] d2[0.250] g[158.801]
>466, d1[0.354] d2[0.302] g[222.858]
>467, d1[0.184] d2[0.261] g[174.216]
>468, d1[0.571] d2[0.188] g[302.184]
>469, d1[0.457] d2[0.748] g[30.356]
>470, d1[0.389] d2[0.383] g[131.296]
>471, d1[0.316] d2[0.470] g[403.916]
>472, d1[0.271] d2[0.295] g[199.027]
>473, d1[0.628] d2[0.176] g[232.175]
>474, d1[0.208] d2[0.600] g[140.025]
>475, d1[0.242] d2[0.167] g[98.645]
>476, d1[0.140] d2[0.196] g[196.060]
>477, 

>674, d1[0.489] d2[0.305] g[172.084]
>675, d1[0.360] d2[0.214] g[112.987]
>676, d1[0.662] d2[0.307] g[254.475]
>677, d1[0.338] d2[0.217] g[126.274]
>678, d1[0.223] d2[0.227] g[54.892]
>679, d1[0.601] d2[0.420] g[128.950]
>680, d1[0.177] d2[0.368] g[325.984]
>681, d1[0.489] d2[0.424] g[339.429]
>682, d1[0.515] d2[0.590] g[36.820]
>683, d1[0.208] d2[0.303] g[444.715]
>684, d1[0.469] d2[0.256] g[106.876]
>685, d1[0.262] d2[0.320] g[204.054]
>686, d1[0.201] d2[0.295] g[112.461]
>687, d1[0.484] d2[0.343] g[65.638]
>688, d1[0.304] d2[0.387] g[58.425]
>689, d1[0.370] d2[0.361] g[107.978]
>690, d1[0.259] d2[0.304] g[554.028]
>691, d1[0.331] d2[0.461] g[69.989]
>692, d1[0.432] d2[0.380] g[261.766]
>693, d1[0.270] d2[0.311] g[150.218]
>694, d1[0.207] d2[0.459] g[404.088]
>695, d1[0.307] d2[0.480] g[91.591]
>696, d1[0.088] d2[0.477] g[326.743]
>697, d1[0.410] d2[0.383] g[292.049]
>698, d1[0.329] d2[0.263] g[92.598]
>699, d1[0.392] d2[0.368] g[48.441]
>700, d1[0.435] d2[0.173] g[279.833]
>701, d1[

>898, d1[0.207] d2[0.309] g[338.785]
>899, d1[0.309] d2[0.492] g[222.752]
>900, d1[0.367] d2[0.423] g[206.276]
>901, d1[0.361] d2[0.496] g[139.265]
>902, d1[0.283] d2[0.504] g[178.553]
>903, d1[0.347] d2[0.205] g[296.728]
>904, d1[0.355] d2[0.239] g[99.749]
>905, d1[0.318] d2[0.775] g[430.240]
>906, d1[0.429] d2[0.348] g[190.268]
>907, d1[0.328] d2[0.330] g[161.545]
>908, d1[0.313] d2[0.411] g[187.240]
>909, d1[0.365] d2[0.445] g[364.641]
>910, d1[0.340] d2[0.289] g[182.792]
>911, d1[0.581] d2[0.424] g[200.875]
>912, d1[0.423] d2[0.419] g[212.092]
>913, d1[0.508] d2[0.504] g[237.972]
>914, d1[0.397] d2[0.320] g[161.335]
>915, d1[0.321] d2[0.344] g[340.562]
>916, d1[0.318] d2[0.302] g[62.795]
>917, d1[0.113] d2[0.414] g[397.761]
>918, d1[0.327] d2[0.489] g[238.977]
>919, d1[0.543] d2[0.454] g[213.966]
>920, d1[0.163] d2[0.352] g[148.453]
>921, d1[0.393] d2[0.284] g[190.948]
>922, d1[0.336] d2[0.287] g[373.985]
>923, d1[0.453] d2[0.209] g[115.329]
>924, d1[0.201] d2[0.232] g[278.407]
>92

...

>40266, d1[0.351] d2[0.389] g[30.018]
>40267, d1[0.322] d2[0.339] g[30.015]
>40268, d1[0.355] d2[0.348] g[35.001]
>40269, d1[0.391] d2[0.403] g[35.267]
>40270, d1[0.361] d2[0.353] g[10.132]
>40271, d1[0.320] d2[0.374] g[65.358]
>40272, d1[0.308] d2[0.370] g[230.614]
>40273, d1[0.336] d2[0.342] g[93.432]
>40274, d1[0.315] d2[0.324] g[169.462]
>40275, d1[0.344] d2[0.388] g[84.558]
>40276, d1[0.330] d2[0.356] g[17.588]
>40277, d1[0.341] d2[0.332] g[46.137]
>40278, d1[0.322] d2[0.324] g[107.051]
>40279, d1[0.371] d2[0.407] g[32.128]
>40280, d1[0.298] d2[0.362] g[38.481]
>40281, d1[0.369] d2[0.382] g[18.556]
>40282, d1[0.378] d2[0.367] g[88.310]
>40283, d1[0.301] d2[0.342] g[30.521]
>40284, d1[0.335] d2[0.368] g[37.333]
>40285, d1[0.340] d2[0.402] g[63.095]
>40286, d1[0.397] d2[0.365] g[57.447]
>40287, d1[0.309] d2[0.339] g[18.231]
>40288, d1[0.352] d2[0.377] g[114.683]
>40289, d1[0.335] d2[0.339] g[68.264]
>40290, d1[0.313] d2[0.412] g[26.892]
>40291, d1[0.312] d2[0.372] g[109.690]
>40292,

>40482, d1[0.427] d2[0.404] g[82.785]
>40483, d1[0.415] d2[0.380] g[66.489]
>40484, d1[0.323] d2[0.348] g[62.572]
>40485, d1[0.330] d2[0.300] g[16.874]
>40486, d1[0.311] d2[0.387] g[35.158]
>40487, d1[0.298] d2[0.348] g[14.931]
>40488, d1[0.368] d2[0.372] g[108.113]
>40489, d1[0.353] d2[0.379] g[98.637]
>40490, d1[0.349] d2[0.328] g[11.072]
>40491, d1[0.334] d2[0.380] g[18.794]
>40492, d1[0.322] d2[0.343] g[106.903]
>40493, d1[0.318] d2[0.306] g[80.335]
>40494, d1[0.368] d2[0.341] g[41.010]
>40495, d1[0.319] d2[0.323] g[2.404]
>40496, d1[0.335] d2[0.317] g[8.079]
>40497, d1[0.315] d2[0.303] g[98.266]
>40498, d1[0.359] d2[0.422] g[96.231]
>40499, d1[0.305] d2[0.337] g[17.916]
>40500, d1[0.365] d2[0.309] g[94.165]
>40501, d1[0.370] d2[0.395] g[60.096]
>40502, d1[0.366] d2[0.338] g[47.228]
>40503, d1[0.313] d2[0.296] g[35.542]
>40504, d1[0.365] d2[0.360] g[28.280]
>40505, d1[0.307] d2[0.400] g[51.702]
>40506, d1[0.379] d2[0.334] g[12.898]
>40507, d1[0.360] d2[0.342] g[25.541]
>40508, d1[0

>40699, d1[0.362] d2[0.331] g[3.148]
>40700, d1[0.353] d2[0.366] g[47.002]
>40701, d1[0.332] d2[0.301] g[78.541]
>40702, d1[0.347] d2[0.371] g[57.006]
>40703, d1[0.379] d2[0.345] g[22.464]
>40704, d1[0.404] d2[0.379] g[42.363]
>40705, d1[0.337] d2[0.343] g[71.133]
>40706, d1[0.318] d2[0.337] g[32.676]
>40707, d1[0.379] d2[0.384] g[17.737]
>40708, d1[0.339] d2[0.349] g[132.577]
>40709, d1[0.335] d2[0.314] g[27.073]
>40710, d1[0.368] d2[0.325] g[22.774]
>40711, d1[0.353] d2[0.310] g[147.037]
>40712, d1[0.313] d2[0.361] g[74.625]
>40713, d1[0.322] d2[0.344] g[247.132]
>40714, d1[0.346] d2[0.407] g[58.005]
>40715, d1[0.364] d2[0.293] g[58.834]
>40716, d1[0.349] d2[0.388] g[23.402]
>40717, d1[0.374] d2[0.320] g[108.271]
>40718, d1[0.310] d2[0.299] g[17.762]
>40719, d1[0.371] d2[0.411] g[35.970]
>40720, d1[0.313] d2[0.347] g[50.872]
>40721, d1[0.375] d2[0.346] g[15.548]
>40722, d1[0.375] d2[0.383] g[16.512]
>40723, d1[0.298] d2[0.362] g[162.880]
>40724, d1[0.406] d2[0.316] g[46.262]
>40725, 

>40915, d1[0.397] d2[0.359] g[28.319]
>40916, d1[0.326] d2[0.382] g[41.818]
>40917, d1[0.370] d2[0.304] g[74.659]
>40918, d1[0.314] d2[0.356] g[32.268]
>40919, d1[0.335] d2[0.342] g[16.643]
>40920, d1[0.394] d2[0.326] g[22.986]
>40921, d1[0.347] d2[0.382] g[94.543]
>40922, d1[0.312] d2[0.369] g[68.129]
>40923, d1[0.335] d2[0.326] g[196.439]
>40924, d1[0.369] d2[0.349] g[20.710]
>40925, d1[0.351] d2[0.291] g[41.161]
>40926, d1[0.345] d2[0.334] g[11.541]
>40927, d1[0.322] d2[0.393] g[121.654]
>40928, d1[0.344] d2[0.336] g[21.138]
>40929, d1[0.320] d2[0.324] g[73.184]
>40930, d1[0.347] d2[0.320] g[23.444]
>40931, d1[0.354] d2[0.335] g[1.233]
>40932, d1[0.364] d2[0.336] g[94.069]
>40933, d1[0.321] d2[0.329] g[50.815]
>40934, d1[0.327] d2[0.373] g[19.747]
>40935, d1[0.360] d2[0.329] g[63.098]
>40936, d1[0.355] d2[0.335] g[39.297]
>40937, d1[0.319] d2[0.356] g[37.297]
>40938, d1[0.337] d2[0.381] g[29.263]
>40939, d1[0.331] d2[0.350] g[30.484]
>40940, d1[0.348] d2[0.366] g[58.057]
>40941, d1[

>41131, d1[0.349] d2[0.350] g[18.303]
>41132, d1[0.314] d2[0.311] g[34.289]
>41133, d1[0.363] d2[0.330] g[82.838]
>41134, d1[0.369] d2[0.346] g[72.107]
>41135, d1[0.364] d2[0.351] g[107.467]
>41136, d1[0.397] d2[0.335] g[29.313]
>41137, d1[0.347] d2[0.346] g[66.425]
>41138, d1[0.318] d2[0.333] g[103.628]
>41139, d1[0.365] d2[0.324] g[15.282]
>41140, d1[0.397] d2[0.315] g[45.230]
>41141, d1[0.340] d2[0.316] g[50.115]
>41142, d1[0.314] d2[0.387] g[56.357]
>41143, d1[0.367] d2[0.382] g[54.349]
>41144, d1[0.343] d2[0.410] g[53.895]
>41145, d1[0.304] d2[0.332] g[56.293]
>41146, d1[0.370] d2[0.324] g[110.817]
>41147, d1[0.363] d2[0.383] g[142.039]
>41148, d1[0.326] d2[0.354] g[208.210]
>41149, d1[0.356] d2[0.385] g[39.624]
>41150, d1[0.405] d2[0.374] g[129.076]
>41151, d1[0.378] d2[0.291] g[37.359]
>41152, d1[0.414] d2[0.353] g[1.918]
>41153, d1[0.335] d2[0.370] g[74.753]
>41154, d1[0.362] d2[0.330] g[22.423]
>41155, d1[0.316] d2[0.356] g[37.341]
>41156, d1[0.374] d2[0.316] g[63.650]
>41157,

In [ ]:
#end the tensorflow session
from tensorflow.keras import backend as K
K.clear_session()

In [ ]:
#for cuda to release GPU memory
from numba import cuda
cuda.select_device(0)
cuda.close()